In [202]:
import pandas as pd
import numpy as np

In [203]:
import keras

In [224]:
data_c = pd.read_csv("data/final/1min/BTC.csv")
data_c.drop("system_time", axis = 1)
data_c['Next_mid'] = data_c.midpoint.shift(-1)
data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

data_c['y'] = 0
data_c.loc[(data_c['Price_move'] < -0.0005), 'y'] = 1
data_c.loc[(data_c['Price_move'] > 0.0005), 'y'] = 2
data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)

data_c.head()

,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,...,PreAvg,JV,PJ,SV,ASV,RQ,RQTri,RQQuad,NV,y
0,0.01,8.243388e+05,854362.503980,-8.890902e-08,-0.000002,-0.000042,-0.000169,-0.000191,-0.000252,-0.000443,...,0.001336,0.0,0.0,7.677271e-08,0.000003,0.000016,4.678313e-07,2.214260e-08,5398.546583,0
1,0.01,4.043096e+05,174798.367015,-8.893365e-08,-0.000030,-0.000050,-0.000206,-0.000216,-0.000250,-0.000256,...,0.001315,0.0,0.0,3.075970e-07,0.000003,0.000016,4.678492e-07,2.212838e-08,5318.152883,2
2,0.01,7.976876e+05,245080.699183,-8.888435e-08,-0.000043,-0.000044,-0.000065,-0.000066,-0.000085,-0.000160,...,0.001217,0.0,0.0,4.481803e-06,0.000003,0.000016,4.508335e-07,2.206376e-08,5320.068883,2
3,6.30,8.938642e+05,504274.642389,-5.587871e-05,-0.000091,-0.000128,-0.000252,-0.000259,-0.000265,-0.000267,...,0.001235,0.0,0.0,1.234156e-06,0.000003,0.000016,4.508341e-07,2.210425e-08,4994.242080,2
4,2.28,1.904225e+06,765199.134667,-2.020032e-05,-0.000107,-0.000107,-0.000158,-0.000189,-0.000189,-0.000235,...,0.001181,0.0,0.0,1.186346e-06,0.000003,0.000016,4.508435e-07,2.215744e-08,5112.644874,1


In [205]:
my_tab = pd.crosstab(index=data_c["y"],  # Make a crosstab
                     columns="count")                  # Name the count column

my_tab

col_0,count
y,
0,9143
1,3893
2,4012


In [206]:
Y = data_c.pop("y")

In [225]:

#data_c = data_c.iloc[:,1:153]
bids = data_c.iloc[:,0:18]

In [226]:
asks = data_c.iloc[:,78:93]

In [243]:
data_bid_ask  = pd.concat([bids.reset_index(drop=True), asks], axis=1)

In [228]:
r, x = data_bid_ask.shape

In [244]:
data_std = data_bid_ask

In [245]:
for i in range(240, r):
    for j in range(x):
        m = data_bid_ask.iloc[0:i,j].mean()
        s = data_bid_ask.iloc[0:i,j].std()
        data_std.iloc[i,j] = (data_bid_ask.iloc[i,j] - m) / s

In [246]:
data_std = data_std.iloc[240:,:].reset_index(drop = True)

In [248]:
data_std.to_csv("data/final/1min/BidAsk/BTC_std.csv", index = False)

In [233]:
Y_std = Y[240:].reset_index(drop = True)

In [249]:
Y_std.to_csv("data/final/1min/BidAsk/BTC_Y_std.csv", index = False)

In [223]:
data_std

,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,...,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14
0,-0.350136,0.137813,2.694658,0.001403,0.004423,0.003951,0.000902,0.001909,0.005307,0.001097,...,0.001434,0.000249,0.008606,0.008307,0.006875,0.009781,0.010536,0.006221,0.003872,0.002697
1,-0.349298,0.249144,-0.093552,0.001210,0.003813,0.004811,0.006013,0.005425,0.006967,0.008930,...,-0.003009,0.000549,-0.001898,-0.002078,-0.001918,0.000451,0.003904,0.004329,0.002598,0.002059
2,-0.348464,0.084839,-0.008091,0.001044,0.005133,0.005023,0.006049,0.002529,0.000496,0.001340,...,0.005264,0.003941,0.005406,0.005951,0.007306,0.003456,0.002696,0.002104,0.002360,0.002691
3,-0.347634,0.136934,0.592426,0.000901,0.002984,0.002321,0.003243,0.003355,0.003678,0.005188,...,-0.004786,-0.006750,-0.003976,-0.004455,-0.003838,-0.004289,-0.006749,-0.003697,-0.000247,-0.000105
4,-0.346807,0.594445,3.735581,0.000777,-0.025079,-0.017018,-0.020611,-0.019608,-0.028487,-0.026107,...,-0.010591,-0.011882,-0.014680,-0.012907,-0.008647,-0.005754,-0.007338,-0.005958,-0.004782,-0.003305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,-0.234392,1.191716,1.859431,0.001226,0.001130,-0.003747,-0.002824,-0.003443,-0.006208,-0.005195,...,0.000042,0.003113,0.001377,0.002654,0.003937,0.009551,0.008093,0.006677,0.005893,0.008583
16564,-0.234378,1.654534,0.839026,0.001214,0.003382,-0.007434,-0.006314,-0.004674,-0.004433,-0.004482,...,0.003697,0.004212,0.003778,0.003474,0.003143,0.004521,0.005485,0.004116,0.005588,0.005243
16565,2.723757,0.958572,2.713586,-0.007800,-0.000358,0.001995,0.001876,0.002980,0.004241,0.002261,...,-0.004161,-0.004701,-0.005988,-0.004201,-0.005354,-0.006599,-0.003196,-0.003843,-0.003939,-0.003135
16566,0.925270,1.278939,0.473249,-0.002250,0.003275,-0.000266,0.001959,0.003141,0.003612,-0.000969,...,-0.004394,-0.001515,-0.001562,-0.002746,-0.003779,-0.004319,-0.005232,-0.005832,-0.006957,-0.007334
